In [7]:
import pandas as pd
from pathlib import Path
from datetime import datetime
import numpy as np

In [12]:
run_folder = Path("../runs").resolve()
runs = {id: run for id, run in enumerate(run_folder.iterdir()) if run.is_dir()}
print(runs)

{0: WindowsPath('D:/00Repos/GPT-Swarm/runs/run_strong_password_checker_2023-04-15_12-20-27')}


In [22]:
def load_run(runs, run_id=None):
    """Logs format:
    {"time": "2023-04-15 12:20:27,477", "level": "INFO", "agent_id": -1, "cycle": -1, "step": "swarm", "message": "Created 2 agents with roles: ['python developer' 'python developer']"}
    {"time": "2023-04-15 12:20:27,477", "level": "INFO", "agent_id": -1, "cycle": -1, "step": "swarm", "message": "Agents roles:\n[['p' 'p']]"} 
    """
    if run_id is None:
        # take the last run
        run_strs = [str(run) for run in runs.values()]
        run_dates = ["_".join(run_str.split("_")[-2:]) for run_str in run_strs]
        run_dates = [datetime.strptime(date, "%Y-%m-%d_%H-%M-%S") for date in run_dates]
        run_id = np.argmax(run_dates)
        run = runs[run_id]
    else:
        run = runs[run_id]
    print(f"Loading run {run_id}: {run}")

    log_file = run / "swarm.json"

    # parse in pandas
    df = pd.read_json(log_file, lines=True)
    df["time"] = pd.to_datetime(df["time"])
    df["agent_id"] = df["agent_id"].astype(int)
    df["cycle"] = df["cycle"].astype(int)
    df["step"] = df["step"].astype(str)
    df["message"] = df["message"].astype(str)
    df["level"] = df["level"].astype(str)

    # unique agents
    agents = df["agent_id"].unique()
    print(f"Found {len(agents)} agents: {agents}")

    # unique cycles
    cycles = df["cycle"].unique()
    print(f"Found {len(cycles)} cycles: {cycles}")

    # unique steps
    steps = df["step"].unique()
    print(f"Found {len(steps)} steps: {steps}")
    return df

def pretty_display(df, agent_id=None, cycle=None, step=None, message=None):
    """Pretty display of the logs"""
    if agent_id is not None:
        df = df[df["agent_id"] == agent_id]
    if cycle is not None:
        df = df[df["cycle"] == cycle]
    if step is not None:
        df = df[df["step"] == step]
    if message is not None:
        df = df[df["message"] == message]
    
    for _, row in df.iterrows():
        print("====================================================================================================================================================")
        print("====================================================================================================================================================")
        print("====================================================================================================================================================")
        print("====================================================================================================================================================")
        print(f"{row['time']} {row['level']} {row['agent_id']} {row['cycle']} {row['step']}")
        print(row['message'])
              

In [23]:
df = load_run(runs)

Loading run 0: D:\00Repos\GPT-Swarm\runs\run_strong_password_checker_2023-04-15_12-20-27
Found 3 agents: [-1  0  1]
Found 3 cycles: [-1  0  1]
Found 4 steps: ['swarm' 'perform_task' 'summarize_incoming_messages' 'call_model']


C:\Users\nicel\AppData\Local\Temp\ipykernel_65576\2795268920.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["time"] = pd.to_datetime(df["time"])


In [24]:
pretty_display(df, agent_id=0)

2023-04-15 12:20:27.478000 INFO 0 0 perform_task
Performing the task
2023-04-15 12:20:27.479000 DEBUG 0 0 summarize_incoming_messages
No incoming messages to summarize.
2023-04-15 12:21:23.159000 DEBUG 0 0 call_model
Condencing the evaluation for the worker 0. 

 Conent: Problem: A password is considered strong if the below conditions are all met:
- It has at least 6 characters and at most 20 characters.
- It contains at least one lowercase letter, at least one uppercase letter, and at least one digit.
- It does not contain three repeating characters in a row (i.e., "Baaabb0" is weak, but "Baaba0" is strong).

Given a string password, return the minimum number of steps required to make password strong. if password is already strong, return 0.

In one step, you can:
- Insert one character to password,
- Delete one character from password, or
- Replace one character of password with another character.
 

Example 1:
Input: password = "a"
Output: 5

Example 2:
Input: password = "aA1"
Outpu